# RAG without Contextual Retrieval

이번에는 Anthropic의 Contextual Retrieval의 개념을 이용해서 Chunk에 Parent doc을 머지해서 테스트를 진행해봤습니다.

아래의 Component를 사용했습니다.
- Framework: Langchain
- LLM: Bedrock Claude
- Embedding: Amazon Titan
- Vector Store: ChromaDB

# Configuring Environment

In [1]:
!pip install langchain

In [2]:
!pip install -U langchain-community

In [3]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.chat_models import BedrockChat
import boto3

# AWS Bedrock Client Connection with boto3

In [4]:
!pip install termcolor

In [5]:
!pip install langchain_aws

In [6]:
import json
import boto3
from pprint import pprint
from termcolor import colored

bedrock_client = boto3.client(service_name='bedrock-runtime')

In [7]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler### LLM 로딩 (Claude-v3-sonnet)

from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_text = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 2048,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fab78f27a90>], client=<botocore.client.BedrockRuntime object at 0x7fab78f25600>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'stop_sequences': ['\n\nHuman']}, streaming=True, max_tokens=2048)

In [8]:
from langchain.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    client=bedrock_client,
    model_id="Titan-Embeddings-G1"
)
dimension = 1536
print("Bedrock Embeddings Model Loaded")

Bedrock Embeddings Model Loaded


/tmp/ipykernel_9406/3925743753.py:3: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  llm_emb = BedrockEmbeddings(


# PDF Files Loading with PyMuPDF

In [9]:
!pip install pymupdf4llm

In [10]:
!wget https://byekang-share-materials.s3.ap-northeast-2.amazonaws.com/file-public-share/Amazon-EC2-UG-Kor-20-30.pdf -O Amazon-EC2-UG-Kor-20-30.pdf

--2024-10-12 09:23:43--  https://byekang-share-materials.s3.ap-northeast-2.amazonaws.com/file-public-share/Amazon-EC2-UG-Kor-20-30.pdf
Resolving byekang-share-materials.s3.ap-northeast-2.amazonaws.com (byekang-share-materials.s3.ap-northeast-2.amazonaws.com)... 52.219.206.54, 3.5.143.36, 52.219.204.74, ...
Connecting to byekang-share-materials.s3.ap-northeast-2.amazonaws.com (byekang-share-materials.s3.ap-northeast-2.amazonaws.com)|52.219.206.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236300 (231K) [application/pdf]
Saving to: ‘Amazon-EC2-UG-Kor-20-30.pdf’

100%[======================================>] 236,300      454KB/s   in 0.5s   

2024-10-12 09:23:44 (454 KB/s) - ‘Amazon-EC2-UG-Kor-20-30.pdf’ saved [236300/236300]



### PDF를 파싱해서 Markdown file로 변환합니다.

In [11]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("Amazon-EC2-UG-Kor-20-30.pdf")

### Markdown file로 잘 변화되었는지를 확인하기 위해서 화면으로 display해 봅니다.

In [12]:
from IPython.display import Markdown, display

display(Markdown(md_text))

# Amazon EC2란 무엇인가요?

Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)
하여 월간 또는 연간 프로세스 또는 웹 사이트 트래픽 급증 등 컴퓨팅 사용량이 많은 작업을 처리할 수
있습니다. 사용량이 감소하면 용량을 다시 축소(스케일 다운)할 수 있습니다.

EC2 인스턴스는 AWS 클라우드의 가상 서버입니다. EC2 인스턴스를 시작할 때 지정하는 인스턴스 유
형에 따라 인스턴스에 사용할 수 있는 하드웨어가 결정됩니다. 인스턴스 유형마다 서로 다른 컴퓨팅,
메모리, 네트워크 및 스토리지 리소스의 균형을 제공합니다. 자세한 내용은 Amazon EC2 인스턴스 유
형 안내서 를 참조하세요.

## Amazon EC2의 기능

Amazon EC2는 다음의 대략적인 기능을 제공합니다.

인스턴스

가상 서버.

Amazon Machine Images (AMIs)

서버에 필요한 구성 요소(운영 체제와 추가 소프트웨어 포함)를 패키징하는 인스턴스용 사전 구성
템플릿.

인스턴스 타입

인스턴스의 다양한 CPU, 메모리, 스토리지, 네트워킹 용량 및 그래픽 하드웨어 구성.

Amazon EBS 볼륨

Amazon Elastic Block Store(Amazon EBS)를 사용하는 데이터에 대한 영구 스토리지 볼륨.

특성 1


-----

인스턴스 스토어 볼륨

인스턴스를 중단, 최대 절전 모드로 전환 또는 종료할 때 삭제되는 임시 데이터용 스토리지 볼륨.

키 페어

인스턴스에 대한 보안 로그인 정보. AWS는 퍼블릭 키를 저장하고 사용자는 프라이빗 키를 안전한
장소에 저장합니다.

보안 그룹

인스턴스에 도달할 수 있는 프로토콜, 포트 및 소스 IP 범위와 인스턴스가 연결할 수 있는 대상 IP
범위를 지정할 수 있는 가상 방화벽.

Amazon EC2는 전자 상거래 웹사이트 운영자 또는 서비스 공급자에 의한 신용카드 데이터의 처리, 저
장 및 전송을 지원하며, 지불 카드(PCI) 보안 표준(DSS)을 준수하는 것으로 검증되었습니다. AWS PCI
규정 준수 패키지의 사본을 요청하는 방법 등 PCI DSS에 대해 자세히 알아보려면 PCI DSS 레벨 1 을
참조하세요.

## 관련 서비스

Amazon EC2와 함께 사용할 서비스

Amazon EC2를 사용하여 배포한 인스턴스와 함께 다른 AWS 서비스를 사용할 수 있습니다.

Amazon EC2 Auto Scaling

애플리케이션의 로드를 처리할 수 있는 정확한 수의 Amazon EC2 인스턴스를 유지하는 데 도움이
됩니다.

AWS Backup

Amazon EC2 인스턴스와 여기에 연결된 Amazon EBS 볼륨의 백업을 자동화합니다.

Amazon CloudWatch

인스턴스와 Amazon EBS 볼륨을 모니터링합니다.

Elastic Load Balancing

수신되는 애플리케이션 트래픽을 여러 인스턴스로 자동 분산합니다.

Amazon GuardDuty

EC2 인스턴스에 대한 잠재적 무단 사용 또는 악의적 사용을 탐지합니다.

관련 서비스 2


-----

EC2 Image Builder

사용자 지정되어 안전하고 최신 상태인 서버 이미지의 생성, 관리 및 배포를 자동화합니다.

AWS Launch Wizard

개별 AWS 리소스를 수동으로 식별하고 프로비저닝할 필요 없이 타사 애플리케이션에 대한 AWS
리소스를 크기 조정, 구성 및 배포합니다.

AWS Systems Manager

이 안전한 엔드 투 엔드 관리 솔루션을 사용하여 EC2 인스턴스에서 대규모로 작업을 수행할 수 있
습니다.

추가 컴퓨팅 서비스

Amazon EC2를 사용하는 대신 다른 AWS 컴퓨팅 서비스를 사용하여 인스턴스를 시작할 수 있습니다.

Amazon Lightsail

저렴하고 예측 가능한 월별 요금으로 프로젝트를 신속하게 배포하는 데 필요한 리소스를 제공하는
클라우드 플랫폼인 Amazon Lightsail을 사용하여 웹 사이트 또는 웹 애플리케이션을 구축합니다.
Amazon EC2와 Lightsail을 비교하려면 Amazon Lightsail 또는 Amazon EC2 섹션을 참조하세요.

Amazon Elastic Container Service(Amazon ECS)

EC2 인스턴스 클러스터에서 컨테이너화된 애플리케이션을 배포, 관리하고 규모를 조정합니다. 자
세한 내용은 AWS 컨테이너 서비스 선택 을 참조하세요.

Amazon Elastic Kubernetes Service(Amazon EKS)

AWS에서 Kubernetes 애플리케이션을 실행합니다. 자세한 내용은 AWS 컨테이너 서비스 선택 을
참조하세요.

## Amazon EC2 액세스

다음 인터페이스를 사용하여 Amazon EC2 인스턴스를 생성하고 관리할 수 있습니다.

Amazon EC2 콘솔

Amazon EC2 인스턴스 및 리소스를 생성하고 관리하는 간단한 웹 인터페이스입니다. AWS 계정
에 가입한 고객은 AWS Management Console에 로그인한 후 콘솔 홈페이지에서 EC2를 선택하여
Amazon EC2에 액세스할 수 있습니다.

EC2 액세스 3


-----

AWS Command Line Interface

명령줄 쉘의 명령을 사용하여 AWS 서비스와 상호 작용할 수 있습니다. Windows, Mac, Linux에서
지원됩니다. AWS CLI에 대한 자세한 내용은 AWS Command Line Interface 사용 설명서 를 참조하
세요. AWS CLI 명령 참조 에서 Amazon EC2 명령을 찾아볼 수 있습니다.

AWS CloudFormation

Amazon EC2는 AWS CloudFormation을 사용한 리소스 생성을 지원합니다. AWS 리소스를 설명
하는 템플릿(JSON 또는 YAML 형식)을 생성하면 AWS CloudFormation에서 이러한 리소스를 프로
비저닝하고 구성합니다. CloudFormation 템플릿을 재사용하여 동일한 리전 및 계정 또는 여러 리
전 및 계정에 동일한 리소스를 여러 번 프로비저닝할 수 있습니다. Amazon EC2에서 지원되는 리소
스 유형 및 속성에 대한 자세한 내용은 AWS CloudFormation 사용 설명서에서 EC2 리소스 유형 참
조 를 참조하세요.

AWS SDK

HTTP나 HTTPS 요청을 직접 보내는 대신, 각 언어가 제공하는 고유의 API를 사용하여 애플리케이
션을 빌드하는 것을 선호하는 개발자를 위해 AWS는, 라이브러리, 샘플 코드, 자습서 및 기타 리소
스를 제공합니다. 이러한 라이브러리는 요청 암호화 서명, 요청 재시도, 오류 응답 처리와 같은 작업
을 자동화하는 기본 기능을 제공하므로 더 쉽게 시작할 수 있습니다. 자세한 내용은 AWS 에서의 구
축을 위한 도구 를 참조하세요.

AWS Tools for PowerShell

AWS SDK for .NET에서 공개하는 기능을 기반으로 하는 PowerShell 모듈 세트입니다. PowerShell
용 도구를 사용하면 PowerShell 명령줄에서 AWS 리소스에 대한 작업을 스크립팅할 수 있습니다.
시작하려면 AWS Tools for Windows PowerShell 사용 설명서 를 참조하십시오. Amazon EC2용
cmdlet은 AWS Tools for PowerShell cmdlet 참조 에서 찾아볼 수 있습니다.

Query API

Amazon EC2에서는 쿼리 API를 제공합니다. 이러한 요청은 HTTP나 HTTPS의 메시지 교환 방식인

GET이나 POST이며, 미리 정해진 이름인 " `Action` "을 쿼리 변수로 사용합니다. Amazon EC2에 관
련된 API 작업에 대한 자세한 내용은 Actions (Amazon EC2 API Reference)을 참조하십시오.

## Amazon EC2 요금

Amazon EC2는 다음과 같은 요금 옵션을 제공합니다.

요금 4


-----

프리 티어

Amazon EC2를 무료로 시작할 수 있습니다. 프리 티어 옵션을 살펴보려면 AWS 프리 티어 를 참조
하세요.

온디맨드 인스턴스

장기 약정이나 선결제 없이 초 단위로, 최소 60초 사용한 인스턴스에 대한 요금을 지불하는 방식입
니다.

절감형 플랜

1년 또는 3년 기간 동안 시간당 USD로 일관된 사용량을 약정하여 Amazon EC2 비용을 절감할 수
있습니다.

예약 인스턴스

1년 또는 3년 기간 동안 인스턴스 유형 또는 지역을 포함해 특정 인스턴스 구성을 약정하여
Amazon EC2 비용을 절감할 수 있습니다.

스팟 인스턴스

미사용 EC2 인스턴스를 요청하여 Amazon EC2 비용을 대폭 줄일 수 있습니다.

전용 호스트

온디맨드 또는 절감형 플랜의 일부로 고객 전용 물리적 EC2 서버를 사용하여 비용을 절감합니다.
기존 서버 기반 소프트웨어 라이선스를 사용하면 규정 준수 요구 사항을 충족하는 데 도움을 받을
수 있습니다.

온디맨드 용량 예약

원하는 기간 동안 특정 가용 영역의 EC2 인스턴스에 대해 용량을 예약합니다.

초당 청구

청구서에서 사용하지 않은 분 및 초 단위의 비용을 제거합니다.

Amazon EC2에 관련된 전체적인 요금 및 가격 목록과 구매 모델 관련 추가 정보는 Amazon EC2 요
금 을 참조하세요.

### 추정, 결제 및 비용 최적화

AWS 사용 사례에 대한 예상 비용을 계산하려면 AWS Pricing Calculator 를 사용하세요.

추정, 결제 및 비용 최적화 5


-----

Microsoft 워크로드를 AWS에 배포된 오픈 소스 및 클라우드 네이티브 서비스를 사용하는 최신 아키텍
처로 변환하는 데 드는 비용을 추정하려면 AWS Modernization Calculator for Microsoft Workloads 를
사용하세요.

청구 요금은 AWS Billing and Cost Management 콘솔 의 청구 및 비용 관리 대시보드에서 확인할 수 있
습니다. 청구서에는 요금 내역을 자세하게 확인할 수 있는 사용 보고서 링크가 포함됩니다. AWS 계정
결제에 대한 자세한 내용은 AWS 결제 및 비용 관리 사용 설명서 를 참조하세요.

AWS 결제, 계정 및 이벤트에 관련된 질문은 AWS Support에 문의 하세요.

샘플 프로비저닝된 환경의 비용을 계산하려면 클라우드 경제 센터 를 참조하세요. 프로비저닝된 환경
의 비용을 계산할 때 EBS 볼륨에 대한 스냅샷 스토리지와 같은 부수적인 비용을 포함해야 합니다.

AWS Trusted Advisor 사용을 통해 AWS 환경의 비용, 보안 및 성능을 최적화할 수 있습니다.

AWS Cost Explorer를 사용하면 EC2 인스턴스의 비용과 사용량을 분석할 수 있습니다. 지난 13개월까
지의 데이터를 볼 수 있고 향후 12개월 동안의 지출을 예상할 수 있습니다. 자세한 내용은 AWS Cost
Management 사용 설명서의 AWS Cost Explorer를 사용한 비용 분석 을 참조하세요.

## 리소스

-  Amazon EC2의 기능

-  AWS re:Post

-  AWS Skill Builder

-  AWS Support

-  실습 자습서

-  웹 호스팅

-  AWS 기반 Windows

리소스 6


-----



# MarkDown 파일을 목차별로 split합니다.

https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/

In [13]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [14]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [15]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_text)

In [16]:
md_header_splits

[Document(metadata={'Header 1': 'Amazon EC2란 무엇인가요?'}, page_content='Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨\n드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플\n리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서\n버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)\n하여 월간 또는 연간 프로세스 또는 웹 사이트 트래픽 급증 등 컴퓨팅 사용량이 많은 작업을 처리할 수\n있습니다. 사용량이 감소하면 용량을 다시 축소(스케일 다운)할 수 있습니다.  \nEC2 인스턴스는 AWS 클라우드의 가상 서버입니다. EC2 인스턴스를 시작할 때 지정하는 인스턴스 유\n형에 따라 인스턴스에 사용할 수 있는 하드웨어가 결정됩니다. 인스턴스 유형마다 서로 다른 컴퓨팅,\n메모리, 네트워크 및 스토리지 리소스의 균형을 제공합니다. 자세한 내용은 Amazon EC2 인스턴스 유\n형 안내서 를 참조하세요.'),
 Document(metadata={'Header 1': 'Amazon EC2란 무엇인가요?', 'Header 2': 'Amazon EC2의 기능'}, page_content='Amazon EC2는 다음의 대략적인 기능을 제공합니다.  \n인스턴스  \n가상 서버.  \nAmazon Machine Images (AMIs)  \n서버에 필요한 구성 요소(운영 체제와 추가 소프트웨어 포함)를 패키징하는 인스턴스용 사전 구성\n템플릿.  \n인스턴스 타입  \n인스턴스의 다양한 CPU, 메모리, 스토리지, 네트워킹 용량 및 그래픽 하드웨어 구성.  \nAmazon EBS 볼륨  \nAmazon Elastic Block Store(Amazon EBS)를 사용하는 데이터에 대한 

### 몇개의 docuement로 split되었는지 출력해 봅니다.

In [17]:
for index, doc in enumerate(md_header_splits):
    print("[Header Doc Index]", index, "------------------------")
    print("  Header 1: ", doc.metadata['Header 1'])
    
    if 'Header 2' in doc.metadata :
        print("    Header 2: ", doc.metadata['Header 2'])
    
    print("      [Doc page_content]:", doc.page_content)
    print("")
    print("")

[Header Doc Index] 0 ------------------------
  Header 1:  Amazon EC2란 무엇인가요?
      [Doc page_content]: Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)
하여 월간 또는 연간 프로세스 또는 웹 사이트 트래픽 급증 등 컴퓨팅 사용량이 많은 작업을 처리할 수
있습니다. 사용량이 감소하면 용량을 다시 축소(스케일 다운)할 수 있습니다.  
EC2 인스턴스는 AWS 클라우드의 가상 서버입니다. EC2 인스턴스를 시작할 때 지정하는 인스턴스 유
형에 따라 인스턴스에 사용할 수 있는 하드웨어가 결정됩니다. 인스턴스 유형마다 서로 다른 컴퓨팅,
메모리, 네트워크 및 스토리지 리소스의 균형을 제공합니다. 자세한 내용은 Amazon EC2 인스턴스 유
형 안내서 를 참조하세요.


[Header Doc Index] 1 ------------------------
  Header 1:  Amazon EC2란 무엇인가요?
    Header 2:  Amazon EC2의 기능
      [Doc page_content]: Amazon EC2는 다음의 대략적인 기능을 제공합니다.  
인스턴스  
가상 서버.  
Amazon Machine Images (AMIs)  
서버에 필요한 구성 요소(운영 체제와 추가 소프트웨어 포함)를 패키징하는 인스턴스용 사전 구성
템플릿.  
인스턴스 타입  
인스턴스의 다양한 CPU, 메모리, 스토리지, 네트워킹 용량 및 그래픽 하드웨어 구성.  
Amazon EBS 볼륨  
Am

# Child Chunk 생성
### RecursiveCharacterTextSplitter를 이용해서 RAG로 사용할 수 있도록 child chunk로 생성합니다.

In [18]:
# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

## Contextual Chunk를 생성합니다.
목차별로 나눠진 chunk (parent chunk)의 맨 뒷부분에 child chunk를 머지해서 Contextual Chunk를 생성합니다.
그리고 나서 해당 Chunk를 page_content로 업데이트해서 저장합니다.

In [38]:
splits = []

for index, parent_doc in enumerate(md_header_splits):
    print("[Parent Doc Index]", index, "------------------------")
    print("--> parent_doc.metadata: ", parent_doc.metadata)
    parent_page_content = parent_doc.page_content
    print("--> parent_page_content:", parent_page_content)
    
    child_docs = text_splitter.split_documents([parent_doc])
    
    for child_doc in child_docs:
        original_child_chunk = child_doc.page_content
        contexual_child_chunk = parent_page_content + ' Merge from Here. ' + original_child_chunk
        child_doc.page_content = contexual_child_chunk
        print("----> original_child_chunk: ", original_child_chunk)
        print("----> contexual_child_chunk: ", contexual_child_chunk)
        
    splits = splits + child_docs

[Parent Doc Index] 0 ------------------------
--> parent_doc.metadata:  {'Header 1': 'Amazon EC2란 무엇인가요?'}
--> parent_page_content: Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)
하여 월간 또는 연간 프로세스 또는 웹 사이트 트래픽 급증 등 컴퓨팅 사용량이 많은 작업을 처리할 수
있습니다. 사용량이 감소하면 용량을 다시 축소(스케일 다운)할 수 있습니다.  
EC2 인스턴스는 AWS 클라우드의 가상 서버입니다. EC2 인스턴스를 시작할 때 지정하는 인스턴스 유
형에 따라 인스턴스에 사용할 수 있는 하드웨어가 결정됩니다. 인스턴스 유형마다 서로 다른 컴퓨팅,
메모리, 네트워크 및 스토리지 리소스의 균형을 제공합니다. 자세한 내용은 Amazon EC2 인스턴스 유
형 안내서 를 참조하세요.
----> original_child_chunk:  Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)
----> contexual_child_chunk:  Amazon Elastic Com

In [40]:
#splits

In [41]:
### 몇개의 청크로 생성되었는지 출력해 봅니다.

In [42]:
for index, doc in enumerate(splits):
    print("[Header Doc Index]", index, "------------------------")
    print("  Header 1: ", doc.metadata['Header 1'])
    
    if 'Header 2' in doc.metadata :
        print("    Header 2: ", doc.metadata['Header 2'])
    
    print("      [Doc page_content]:", doc.page_content)
    print("")
    print("")

[Header Doc Index] 0 ------------------------
  Header 1:  Amazon EC2란 무엇인가요?
      [Doc page_content]: Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)
하여 월간 또는 연간 프로세스 또는 웹 사이트 트래픽 급증 등 컴퓨팅 사용량이 많은 작업을 처리할 수
있습니다. 사용량이 감소하면 용량을 다시 축소(스케일 다운)할 수 있습니다.  
EC2 인스턴스는 AWS 클라우드의 가상 서버입니다. EC2 인스턴스를 시작할 때 지정하는 인스턴스 유
형에 따라 인스턴스에 사용할 수 있는 하드웨어가 결정됩니다. 인스턴스 유형마다 서로 다른 컴퓨팅,
메모리, 네트워크 및 스토리지 리소스의 균형을 제공합니다. 자세한 내용은 Amazon EC2 인스턴스 유
형 안내서 를 참조하세요. Merge from Here. Amazon Elastic Compute Cloud(Amazon EC2)는 Amazon Web Services(AWS) 클라우드에서 온디맨
드 확장 가능 컴퓨팅 용량을 제공합니다. Amazon EC2를 사용하면 하드웨어 비용이 절감되므로 애플
리케이션을 더욱 빠르게 개발하고 배포할 수 있습니다. Amazon EC2를 사용하여 원하는 수의 가상 서
버를 구축하고 보안 및 네트워킹을 구성하며 스토리지를 관리할 수 있습니다. 용량을 추가(스케일 업)


[Header Doc Index] 1 ------------------------
  Header 1:  Amazon EC2란 무엇인가요?
      [D

# VectorDB 로 임베딩해서 저장

### 위에서 생성한 child chunk를 embedding해서 Chroma vector store에 저장

In [43]:
!pip install chromadb

In [55]:
vector_store = Chroma.from_documents(documents=splits,
                                     embedding=BedrockEmbeddings()
                                    )

# Q&A Retrieval Chain

In [56]:
document_prompt = PromptTemplate(
        input_variables=["page_content"],
        template="Context:\ncontent:{page_content}\n",
    )

In [57]:
QUERY_PROMPT_TEMPLATE = """\
H:
Answer the question based on the provided context. Do not create false information.
{context}
Question: {question}
A:
"""

In [58]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain

In [59]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(QUERY_PROMPT_TEMPLATE) # prompt_template defined above

In [60]:
llm_chain = LLMChain(llm=llm_text, prompt=QA_CHAIN_PROMPT, callbacks=None, verbose=True)

In [61]:
combine_documents_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
        document_prompt=document_prompt,
        callbacks=None,
    )

In [62]:
qa = RetrievalQA(
        combine_documents_chain=combine_documents_chain,
        callbacks=None,
        verbose=True,
        retriever=vector_store.as_retriever(search_kwargs={'k': 1}),
        return_source_documents=True,
    )

# 질문에 대한 답변 테스트

In [63]:
question = '온디맨드 용량 예약 방식와 예약 인스턴스의 방식의 차이는 무엇입니까?'

In [64]:
res = qa(question)



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:
H:
Answer the question based on the provided context. Do not create false information.
Context:
content:Amazon EC2는 다음과 같은 요금 옵션을 제공합니다.  
요금 4  
-----  
프리 티어  
Amazon EC2를 무료로 시작할 수 있습니다. 프리 티어 옵션을 살펴보려면 AWS 프리 티어 를 참조
하세요.  
온디맨드 인스턴스  
장기 약정이나 선결제 없이 초 단위로, 최소 60초 사용한 인스턴스에 대한 요금을 지불하는 방식입
니다.  
절감형 플랜  
1년 또는 3년 기간 동안 시간당 USD로 일관된 사용량을 약정하여 Amazon EC2 비용을 절감할 수
있습니다.  
예약 인스턴스  
1년 또는 3년 기간 동안 인스턴스 유형 또는 지역을 포함해 특정 인스턴스 구성을 약정하여
Amazon EC2 비용을 절감할 수 있습니다.  
스팟 인스턴스  
미사용 EC2 인스턴스를 요청하여 Amazon EC2 비용을 대폭 줄일 수 있습니다.  
전용 호스트  
온디맨드 또는 절감형 플랜의 일부로 고객 전용 물리적 EC2 서버를 사용하여 비용을 절감합니다.
기존 서버 기반 소프트웨어 라이선스를 사용하면 규정 준수 요구 사항을 충족하는 데 도움을 받을
수 있습니다.  
온디맨드 용량 예약  
원하는 기간 동안 특정 가용 영역의 EC2 인스턴스에 대해 용량을 예약합니다.  
초당 청구  
청구서에서 사용하지 않은 분 및 초 단위의 비용을 제거합니다.  
Amazon EC2에 관련된 전체적인 요금 및 가격 목록과 구매 모델 관련 추가 정보는 Amazon EC2 요
금 을 참조하세요. Merge from Here. 전용 호스트  
온디맨드 또는 절감형 플랜의 일부로 고객 전용 물리적 EC2 서버를 사용하여 비용을

In [66]:
print(res['result'])

온디맨드 용량 예약과 예약 인스턴스의 주요 차이점은 다음과 같습니다:

1. 기간: 
- 온디맨드 용량 예약: 원하는 기간 동안 예약 가능
- 예약 인스턴스: 1년 또는 3년의 고정된 기간 약정

2. 범위:
- 온디맨드 용량 예약: 특정 가용 영역의 EC2 인스턴스에 대해 용량을 예약
- 예약 인스턴스: 인스턴스 유형이나 지역을 포함한 특정 인스턴스 구성을 약정

3. 목적:
- 온디맨드 용량 예약: 주로 용량 확보에 초점
- 예약 인스턴스: 비용 절감에 초점

4. 요금 할인:
- 온디맨드 용량 예약: 명시적인 요금 할인 언급 없음
- 예약 인스턴스: Amazon EC2 비용을 절감할 수 있다고 명시

따라서 온디맨드 용량 예약은 특정 가용 영역에서 용량을 확보하는 데 중점을 두는 반면, 예약 인스턴스는 장기 약정을 통해 비용을 절감하는 데 중점을 둡니다.


# 결론

### Child chunk만 이용했을 때는, "온디맨드 용량 예약"에 대한 정보만 context로 활용되고 있기 때문에 정확한 답변을 하지 못하고 있었으나, Contextual Retrieval 기능을 이용해서 parent content를 머지했을 경우에는 답변이 정확해 지고 있는 것을 확인할 수 있었습니다.